In [93]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [94]:
load_dotenv(override=True)
openai = OpenAI()

In [95]:
reader = PdfReader(r"C:\Users\James Moelling\projects\agents\My_AIProjects\WindowWolfChatbot.pdf")
WindowWolfChatbot = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        WindowWolfChatbot += text

In [96]:
print(WindowWolfChatbot)

James Moelling  
Key Skills & Strengths 
• • Punctual & Reliable – Consistently arrive 10+ minutes early to work. 
• • Customer Service Excellence – Achieved three raises at In-N-Out Burger — two 
specifically for exceptional customer service (A rare distinction for associates, out of nearly 
sixty associates only twelve myself included has these customer service raises). 
• • Problem Solving – Hands-on experience addressing unexpected challenges in property 
maintenance and contracting. 
• • Physical Endurance – Comfortable working long hours in heat; experienced in hauling 
heavy loads, tearing down fences, and general labor. Regular weightlifter and cyclist. 
• • Attention to Detail – Developed a sharp eye at Lone Star Window Cleaning for spotting 
imperfections in windows and learned how important the small details can be. Having this 
attention to detail helps me get the job done on the first run through preventing having to go 
back and do it again. 
• • Adaptable & Quick Learner

In [97]:
# Read the summary file - try multiple path approaches
import os

# Try different path approaches
possible_paths = [
    "My_AIProjects/WindowWolfSummary.txt",  # Relative from project root
    os.path.join(os.getcwd(), "My_AIProjects", "WindowWolfSummary.txt"),  # Absolute
    "WindowWolfSummary.txt",  # If running from My_AIProjects directory
    os.path.join(os.path.dirname(__file__), "WindowWolfSummary.txt") if '__file__' in globals() else None  # If running as script
]

summary = None
for path in possible_paths:
    if path and os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                summary = f.read()
            print(f"Summary loaded successfully from: {path}")
            break
        except Exception as e:
            print(f"Failed to read {path}: {e}")
            continue

if summary is None:
    print("Could not find WindowWolfSummary.txt file. Creating a default summary.")
    summary = "James Moelling is a driven and detail-oriented entrepreneur passionate about quality, technology, and personal growth — blending hands-on skill, strong work ethic, and a love for learning to build reliable, people-focused businesses."

print(f"Summary content: {summary}")

Summary loaded successfully from: WindowWolfSummary.txt
Summary content: My name is James Moelling, I'm driven and detail-oriented entrepreneur passionate about quality, technology, and personal growth — 
blending hands-on skill, strong work ethic, and a love for learning to build reliable, people-focused businesses


In [98]:
name = "James Moelling"
business_name = "Window Wolf"

In [99]:
system_prompt = f"""
    You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s business {business_name} and background, skills and experience. \
    Your responsibility is to represent {name} and {business_name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and Skills profile and how {business_name} operates and what equipment is used\
    which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer, say so."""
    
system_prompt += f"\n\n## Summary:\n{summary}\n\n## WindowWolfChatbot:\n{WindowWolfChatbot}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name} who owns and operates {business_name}."


In [100]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}]

    for entry in history:
        messages.append({"role": "user", "content": entry[0]})
        messages.append({"role": "assistant", "content": entry[1]})

    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [101]:
# Simple version without quality control
with gr.Blocks(title=f"{business_name} - Simple Chatbot") as simple_interface:
    gr.Markdown(f"# {business_name} - Simple Window Cleaning Chatbot")
    gr.Markdown(f"Chat with {business_name}'s AI assistant. Ask about our window cleaning services, pricing, or learn more about our owner {name}!")
    
    chatbot = gr.Chatbot(type="messages", height=400)
    msg = gr.Textbox(label="Your message", placeholder="Ask about our window cleaning services...")
    clear = gr.Button("Clear")
    
    def simple_respond(message, history):
        if not message.strip():
            return history, ""
        
        # Convert messages format to tuple format for our chat function
        tuple_history = []
        for msg in history:
            if msg["role"] == "user":
                tuple_history.append((msg["content"], ""))
            elif msg["role"] == "assistant":
                if tuple_history:
                    tuple_history[-1] = (tuple_history[-1][0], msg["content"])
        
        # Get response from our chat function (using the simple version from cell 7)
        response = chat(message, tuple_history)
        
        # Add to history
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": response})
        
        return history, ""
    
    msg.submit(simple_respond, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: ([], ""), outputs=[chatbot, msg])

simple_interface.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [102]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [103]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
    you are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
    The Agent is playing the role of {name} and is representing {name} and {name}'s business {business_name} on their website. \
    The Agent has been provided with context on {name} and {business_name} in the form of a summary and details on {name}'s skills and experience. \
    and details on {business_name}'s window cleaning services, and operational procedures. \
    Be strict with your evaluation, as the Agent is representing {name} and {business_name} on their website. \
    but allow for some flexibility in the response as to breifly explain the answers. \
    If the Agent's response is not acceptable, provide feedback on how to improve it. \
    If the Agent's response is acceptable, respond with 'is_acceptable': True. \
    If the Agent's response is not acceptable, respond with 'is_acceptable': False and provide feedback on how to improve it. \
    Your response should be in the form of a JSON object with the following fields: \
    'is_acceptable': bool, 'feedback': str"

# Initialize evaluator_user_prompt before appending to it
evaluator_user_prompt = ""
evaluator_user_prompt += f"\n\n## Summary:\n{summary}\n\n## WindowWolfChatbot:\n{WindowWolfChatbot}\n\n"
evaluator_user_prompt += f"With this context, please evaluate the Agent's latest response to the User's question."

In [104]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt


In [105]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

In [106]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = ollama.beta.chat.completions.parse(model=model_name, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [107]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you clean window screens?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [108]:
reply

"Yes, absolutely! I always clean window screens along with the windows. Cleaning the screens helps prevent dirt from dripping back down onto your freshly cleaned windows after it rains. I pay close attention to every detail to ensure that not only do the windows look spotless, but the screens are also clean and free of debris. It's all part of the service to give you the best results possible!"

In [109]:
evaluate(reply, "Do you clean window screens?", [])

Evaluation(is_acceptable=True, feedback='')

In [110]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content
    

In [111]:
def chat(message, history):
    # Convert Gradio history format to OpenAI format
    messages = [{"role": "system", "content": system_prompt}]
    
    # Add conversation history
    for entry in history:
        messages.append({"role": "user", "content": entry[0]})
        messages.append({"role": "assistant", "content": entry[1]})
    
    # Add current user message
    messages.append({"role": "user", "content": message})
    
    # Generate response
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    # Evaluate response
    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print(f"Response accepted")
        return reply
    else:
        print(f"Response rejected")
        print(f"Reason: {evaluation.feedback}")
        # Convert history back to the format expected by rerun function
        history_messages = []
        for entry in history:
            history_messages.append({"role": "user", "content": entry[0]})
            history_messages.append({"role": "assistant", "content": entry[1]})
        return rerun(reply, message, history_messages, evaluation.feedback)

In [ ]:
# Create a custom Gradio interface to avoid ChatInterface issues
def chat_with_history(message, history):
    """Wrapper function for the chat function"""
    return chat(message, history)

# Create the interface manually
with gr.Blocks(title=f"{business_name} - Window Cleaning Services Chatbot") as interface:
    gr.Markdown(f"# {business_name} - Window Cleaning Services Chatbot")
    gr.Markdown(f"Chat with {business_name}'s AI assistant. Ask about our window cleaning services, pricing, or learn more about our owner {name}!")
    
    chatbot = gr.Chatbot(type="messages", height=400)
    msg = gr.Textbox(label="Your message", placeholder="Ask about our window cleaning services...")
    clear = gr.Button("Clear")
    
    def respond(message, history):
        if not message.strip():
            return history, ""
        
        # Convert messages format to tuple format for our chat function
        tuple_history = []
        for msg in history:
            if msg["role"] == "user":
                tuple_history.append((msg["content"], ""))
            elif msg["role"] == "assistant":
                if tuple_history:
                    tuple_history[-1] = (tuple_history[-1][0], msg["content"])
        
        # Get response from our chat function
        response = chat(message, tuple_history)
        
        # Add to history
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": response})
        
        return history, ""
    
    msg.submit(respond, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: ([], ""), outputs=[chatbot, msg])

interface.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Response accepted
Response accepted
